### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [10]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [11]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [12]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [13]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 12, 12, 14, 35, 956670, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent

# Agent 需要設定 System Message，來定義它的角色與行為
# 這裡我們定義一個航空公司的助理，並且要求它回答時要幽默風趣且簡短
agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [ ]:
from autogen_core import CancellationToken

# CancellationToken 用來控制非同步任務的取消
response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Great choice! London has everything from tea time to a monument that looks like a giant needle. Just remember, umbrella in hand, because the weather can be as unpredictable as a toddler in a candy store!'

### Let's make a local database of ticket prices

In [16]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [17]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [18]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [19]:
get_city_price("Rome")

499.0

In [ ]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price], # 將 get_city_price 函式作為工具傳入 Agent
    reflect_on_tool_use=True # 設定 True 讓 Agent 可以反思工具的使用情況
)

# 當 AssistantAgent 的 reflect_on_tool_use 設為 False 時，Agent 在使用工具（例如 get_city_price）# 後，不會進行反思或自我檢查工具使用的過程。這代表：
# Agent 只會直接呼叫工具並回傳結果，不會分析工具使用是否合理或是否需要修正。
# 回應內容通常較直接，不會包含工具使用的思考過程或多層次推理。
# 若工具回傳異常或結果不符預期，Agent 不會主動嘗試其他方法或自我修正。
# 簡單來說，reflect_on_tool_use=False 會讓 Agent 行為更像「工具呼叫者」，而不是「會反思的智能助理」。



In [ ]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content) # 印出Agent使用工具的過程
response.chat_message.content

[FunctionCall(id='call_HQaArGXvtTkfkW0rCVVu4VaS', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_HQaArGXvtTkfkW0rCVVu4VaS', is_error=False)]


'Sure! A roundtrip ticket to London will cost you $299. Just remember, the only thing heavier than your luggage will be your jet lag!'